# Create a SageMaker MLOps Project for Pipelines
Note:  This requires that you have enabled products within SageMaker Studio

![](../img/enable-service-catalog-portfolio-for-studio.png)

In [1]:
import os
import sagemaker
import logging
import boto3
import sagemaker
import pandas as pd
from pprint import pprint

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)
sc = boto3.Session().client(service_name='servicecatalog', region_name=region)
sts = boto3.Session().client(service_name='sts', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
search_response = sc.search_products(
   Filters={
       'FullTextSearch': 
       [
           'MLOps template for model building, training, and deployment'
       ]
   }
)

sagemaker_pipeline_product_id = search_response['ProductViewSummaries'][0]['ProductId']
print(sagemaker_pipeline_product_id)

# pprint(search_response)

prod-j3ufw6hl7utxm


In [3]:
describe_response = sc.describe_product(Id=sagemaker_pipeline_product_id)

sagemaker_pipeline_product_provisioning_artifact_id = describe_response['ProvisioningArtifacts'][0]['Id']

pprint(describe_response)

{'Budgets': [],
 'LaunchPaths': [{'Id': 'lpv2-otfcrq7zdg5sc',
                  'Name': 'Amazon SageMaker Solutions and ML Ops products'}],
 'ProductViewSummary': {'HasDefaultPath': False,
                        'Id': 'prodview-wbmrovteqfhoy',
                        'Name': 'MLOps template for model building, training, '
                                'and deployment',
                        'Owner': 'Amazon SageMaker',
                        'ProductId': 'prod-j3ufw6hl7utxm',
                        'ShortDescription': 'This template enables you to '
                                            'easily build, train, and deploy '
                                            'machine learning models. You can '
                                            'adopt MLOps best practices and '
                                            'enable Continuous '
                                            'Integration/Continuous Deployment '
                                            'for build

In [4]:
print(sagemaker_pipeline_product_provisioning_artifact_id)

pa-oacphmo7m2bji


# Create the Project


In [5]:
import time

timestamp = int(time.time())

In [6]:
sagemaker_project_name = 'dsoaws-{}'.format(timestamp)

create_response = sm.create_project(
    ProjectName=sagemaker_project_name,
    ProjectDescription='dsoaws-{}'.format(timestamp),
    ServiceCatalogProvisioningDetails={
        'ProductId': sagemaker_pipeline_product_id,
        'ProvisioningArtifactId': sagemaker_pipeline_product_provisioning_artifact_id
     }
)

sagemaker_project_id = create_response['ProjectId']
sagemaker_project_arn = create_response['ProjectArn']

pprint(create_response)

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:835319576252:project/dsoaws-1609826255',
 'ProjectId': 'p-mz9w9bo7oooi',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '112',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Tue, 05 Jan 2021 05:57:35 GMT',
                                      'x-amzn-requestid': '248de865-d800-46d2-949b-44b136fbed65'},
                      'HTTPStatusCode': 200,
                      'RequestId': '248de865-d800-46d2-949b-44b136fbed65',
                      'RetryAttempts': 0}}


In [7]:
sagemaker_project_name_and_id = '{}-{}'.format(sagemaker_project_name, sagemaker_project_id)

print(sagemaker_project_name_and_id)

dsoaws-1609826255-p-mz9w9bo7oooi


# Describe the Newly-Created Project

In [8]:
response = sm.describe_project(ProjectName=sagemaker_project_name)
print(response)

{'ProjectArn': 'arn:aws:sagemaker:us-east-1:835319576252:project/dsoaws-1609826255', 'ProjectName': 'dsoaws-1609826255', 'ProjectId': 'p-mz9w9bo7oooi', 'ProjectDescription': 'dsoaws-1609826255', 'ServiceCatalogProvisioningDetails': {'ProductId': 'prod-j3ufw6hl7utxm', 'ProvisioningArtifactId': 'pa-oacphmo7m2bji'}, 'ProjectStatus': 'Pending', 'CreationTime': datetime.datetime(2021, 1, 5, 5, 57, 35, 759000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': '540e7173-a0d9-4dd3-a000-ab3869576fe6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '540e7173-a0d9-4dd3-a000-ab3869576fe6', 'content-type': 'application/x-amz-json-1.1', 'content-length': '360', 'date': 'Tue, 05 Jan 2021 05:57:35 GMT'}, 'RetryAttempts': 0}}


In [9]:
project_status = response['ProjectStatus']
print(project_status)

Pending


# Attach IAM Policies for FeatureStore 
This is used for Code Build Pipeline Executions.

In [10]:
sc_role_name='AmazonSageMakerServiceCatalogProductsUseRole'

In [11]:
account_id = sts.get_caller_identity()['Account']
print(account_id)

835319576252


In [12]:
sc_role_arn = 'arn:aws:iam::{}:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole'.format(account_id)
print(sc_role_arn)

arn:aws:iam::835319576252:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [13]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': 'b0f96772-3e38-40b0-b9dc-60e25dfcb319', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b0f96772-3e38-40b0-b9dc-60e25dfcb319', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Tue, 05 Jan 2021 05:57:36 GMT'}, 'RetryAttempts': 0}}


In [14]:
response = iam.attach_role_policy(
    RoleName=sc_role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFeatureStoreAccess'
)

print(response)

{'ResponseMetadata': {'RequestId': '37af1982-043c-4d71-9ded-2d6977961bc2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '37af1982-043c-4d71-9ded-2d6977961bc2', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Tue, 05 Jan 2021 05:57:36 GMT'}, 'RetryAttempts': 0}}


# _Wait for Project to be Created_

In [15]:
import time

try:
    create_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
    project_status = create_project_response['ProjectStatus']
    print('Creating Project...')

    while project_status in ['Pending', 'CreateInProgress']:
        print('Please wait...')
        time.sleep(30)
        create_project_response = sm.describe_project(ProjectName=sagemaker_project_name)
        project_status = create_project_response['ProjectStatus']
        print('Project status: {}'.format(project_status))

    if project_status == 'CreateCompleted':   
        print('Project {}'.format(project_status))

    else:
        print('Project status: {}'.format(project_status))
        raise Exception('Project not created.')
        
except Exception as e:
    print(e)
    
print(create_project_response)

Creating Project...
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateInProgress
Please wait...
Project status: CreateCompleted
Project CreateCompleted
{'ProjectArn': 'arn:aws:sagemaker:us-east-1:835319576252:project/dsoaws-1609826255', 'ProjectName': 'dsoaws-1609826255', 'ProjectId': 'p-mz9w9bo7oooi', 'ProjectDescription': 'dsoaws-1609826255', 'ServiceCatalogProvisioningDetails': {'ProductId': 'prod-j3ufw6hl7utxm', 'ProvisioningArtifactId': 'pa-oacphmo7m2bji'}, 'ServiceCatalogProvisionedProductDetails': {'ProvisionedProductId': 'pp-nlsshgqldnx5m'}, 'ProjectStatus': 'CreateCompleted', 'CreationTime': datetime.datetime(2021, 1, 5, 5, 57, 35, 759000, tzinfo=tzlocal()), 'ResponseMetadata': {'RequestId': 'd33cd4ca-41fc-437d-96a3-469566430c99', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd33cd4ca-41fc-437d-96a3-469566430c99', 'content-type': 'app

# _Wait for Project to be Created ^^ Above ^^_

# Stop the `Abalone` Sample Pipeline that Ships with SageMaker Pipelines
The sample "abalone" pipeline starts automatically when we create the project.  We want to stop this pipeline to release these resources and use them for our own pipeline.

In [17]:
sample_abalone_pipeline_execution_arn = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries'][0]['PipelineExecutionArn']

print(sample_abalone_pipeline_execution_arn)

arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi/execution/tupernr5vfdh


In [18]:
sm.stop_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)

{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi/execution/tupernr5vfdh',
 'ResponseMetadata': {'RequestId': '4df95727-0bac-4171-b671-144883f97a67',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4df95727-0bac-4171-b671-144883f97a67',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '132',
   'date': 'Tue, 05 Jan 2021 06:00:07 GMT'},
  'RetryAttempts': 0}}

In [29]:
try:
    describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
    pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']

    while pipeline_execution_status not in ['Stopped', 'Failed']:
        print('Please wait...')
        time.sleep(30)
        describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
        pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']
        print('Pipeline execution status: {}'.format(pipeline_execution_status))

    if pipeline_execution_status in ['Stopped', 'Failed']:   
        print('Pipeline execution status {}'.format(pipeline_execution_status))
    else:
        print('Pipeline execution status: {}'.format(pipeline_execution_status))
        raise Exception('Pipeline execution not deleted.')
        
except Exception as e:
    print(e)
    
print(describe_pipeline_execution_response)

Pipeline execution status Stopped
{'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi', 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi/execution/tupernr5vfdh', 'PipelineExecutionDisplayName': 'execution-1609826384700', 'PipelineExecutionStatus': 'Stopped', 'CreationTime': datetime.datetime(2021, 1, 5, 5, 59, 44, 572000, tzinfo=tzlocal()), 'LastModifiedTime': datetime.datetime(2021, 1, 5, 6, 3, 36, 25000, tzinfo=tzlocal()), 'CreatedBy': {}, 'LastModifiedBy': {}, 'ResponseMetadata': {'RequestId': 'b43449e6-65fa-45e8-96e9-d71b9fcb704b', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'b43449e6-65fa-45e8-96e9-d71b9fcb704b', 'content-type': 'application/x-amz-json-1.1', 'content-length': '427', 'date': 'Tue, 05 Jan 2021 06:04:52 GMT'}, 'RetryAttempts': 0}}


In [30]:
sm.delete_pipeline(PipelineName=sagemaker_project_name_and_id)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi',
 'ResponseMetadata': {'RequestId': '5ebacd78-367e-4ca7-8ab7-84e75f753c3a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5ebacd78-367e-4ca7-8ab7-84e75f753c3a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '100',
   'date': 'Tue, 05 Jan 2021 06:04:58 GMT'},
  'RetryAttempts': 0}}

# Clone the MLOps Repositories in AWS CodeCommit

In [31]:
import os

sm_studio_root_path='/root/' 
sm_notebooks_root_path='/home/ec2-user/SageMaker/'

root_path = sm_notebooks_root_path if os.path.isdir(sm_notebooks_root_path) else sm_studio_root_path

print(root_path)

/home/ec2-user/SageMaker/


In [32]:
print(region)

us-east-1


In [33]:
code_commit_repo1 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modelbuild'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo1)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modelbuild


In [34]:
sagemaker_mlops_build_code = '{}{}/sagemaker-{}-modelbuild'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_build_code)

/home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modelbuild


In [35]:
code_commit_repo2 = 'https://git-codecommit.{}.amazonaws.com/v1/repos/sagemaker-{}-modeldeploy'.format(region, sagemaker_project_name_and_id)
print(code_commit_repo2)

https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modeldeploy


In [36]:
sagemaker_mlops_deploy_code = '{}{}/sagemaker-{}-modeldeploy'.format(root_path, sagemaker_project_name_and_id, sagemaker_project_name_and_id)
print(sagemaker_mlops_deploy_code)

/home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modeldeploy


In [37]:
!git config --global credential.helper '!aws codecommit credential-helper $@'
!git config --global credential.UseHttpPath true

# _Wait for Project to be Created ^^ Above ^^_

In [38]:
!git clone $code_commit_repo1 $sagemaker_mlops_build_code

Cloning into '/home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modelbuild'...
remote: Counting objects: 26, done.
Unpacking objects: 100% (26/26), done.


In [39]:
!git clone $code_commit_repo2 $sagemaker_mlops_deploy_code

Cloning into '/home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modeldeploy'...
remote: Counting objects: 12, done.
Unpacking objects: 100% (12/12), done.


# Remove Stock `Abalone` Example Code

In [40]:
!rm -rf $sagemaker_mlops_build_code/pipelines/abalone

# Copy Workshop Code Into Local Project Folders

In [41]:
workshop_project_build_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild'.format(root_path)
print(workshop_project_build_code)

/home/ec2-user/SageMaker/workshop/10_pipeline/sagemaker/sagemaker-project-modelbuild


In [42]:
workshop_project_deploy_code='{}workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy'.format(root_path)
print(workshop_project_deploy_code)

/home/ec2-user/SageMaker/workshop/10_pipeline/sagemaker/sagemaker-project-modeldeploy


In [43]:
!cp -R $workshop_project_build_code/* $sagemaker_mlops_build_code/

In [44]:
!cp -R $workshop_project_deploy_code/* $sagemaker_mlops_deploy_code/

# Commit New Code 

In [45]:
print(sagemaker_mlops_build_code)

/home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modelbuild


In [46]:
!cd $sagemaker_mlops_build_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add/rm <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   codebuild-buildspec.yml
	deleted:    pipelines/abalone/__init__.py
	deleted:    pipelines/abalone/evaluate.py
	deleted:    pipelines/abalone/pipeline.py
	deleted:    pipelines/abalone/preprocess.py
	modified:   pipelines/run_pipeline.py
	modified:   setup.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	pipelines/dsoaws/

no changes added to commit (use "git add" and/or "git commit -a")
[master 0e02d04] Data Science on AWS
 14 files changed, 2227 insertions(+), 452 deletions(-)
 delete mode 100644 pipelines/abalone/evaluate.py
 delete mode 100644 pipelines/abalone/pipeline.py
 delete mode 100644 pipelines/abalone/preprocess.py
 rename pipelines/{abalone => dsoaws}/__init__.py (1

In [47]:
!cd $sagemaker_mlops_deploy_code; git status; git add --all .; git commit -m "Data Science on AWS"; git push

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   README.md
	modified:   prod-config.json
	modified:   staging-config.json

no changes added to commit (use "git add" and/or "git commit -a")
[master 99f41fe] Data Science on AWS
 3 files changed, 3 insertions(+), 3 deletions(-)
Enumerating objects: 9, done.
Counting objects: 100% (9/9), done.
Delta compression using up to 8 threads.
Compressing objects: 100% (5/5), done.
Writing objects: 100% (5/5), 442 bytes | 442.00 KiB/s, done.
Total 5 (delta 4), reused 0 (delta 0)
To https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modeldeploy
   1955a17..99f41fe  master -> master


In [58]:
!ls -al $sagemaker_mlops_build_code/pipelines/dsoaws/

-rw-rw-r-- 1 ec2-user ec2-user 15072 Jan  5 06:05 /home/ec2-user/SageMaker/dsoaws-1609826255-p-mz9w9bo7oooi/sagemaker-dsoaws-1609826255-p-mz9w9bo7oooi-modelbuild/pipelines/dsoaws/pipeline.py


In [59]:
!pygmentize $sagemaker_mlops_build_code/pipelines/dsoaws/pipeline.py

"""
Example workflow pipeline script for BERT pipeline.

                                                 . -RegisterModel
                                                .
    Process-> Train -> (Evaluate -> Condition) .
                                                .
                                                 . -(stop)

Implements a get_pipeline(**kwargs) method.
"""

import os
import boto3
import logging
import time

from botocore.exceptions import ClientError

import sagemaker
import sagemaker.session
from sagemaker.estimator import Estimator
from sagemaker.inputs import TrainingInput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.tensorflow import TensorFlow

from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)

from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)

from

# You Are Now Running a BERT Pipeline!

In [54]:
from pprint import pprint

list_executions_response = sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)['PipelineExecutionSummaries']

pprint(list_executions_response)

# try:
#     describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
#     pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']

#     while pipeline_execution_status not in ['Stopped', 'Failed']:
#         print('Please wait...')
#         time.sleep(30)
#         describe_pipeline_execution_response = sm.describe_pipeline_execution(PipelineExecutionArn=sample_abalone_pipeline_execution_arn)
#         pipeline_execution_status = describe_pipeline_execution_response['PipelineExecutionStatus']
#         print('Pipeline execution status: {}'.format(pipeline_execution_status))

#     if pipeline_execution_status in ['Stopped', 'Failed']:   
#         print('Pipeline execution status {}'.format(pipeline_execution_status))
#     else:
#         print('Pipeline execution status: {}'.format(pipeline_execution_status))
#         raise Exception('Pipeline execution not deleted.')
        
# except Exception as e:
#     print(e)
    
# print(describe_pipeline_execution_response)


[{'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi/execution/42ulswocfppp',
  'PipelineExecutionDisplayName': 'execution-1609827003925',
  'PipelineExecutionStatus': 'Executing',
  'StartTime': datetime.datetime(2021, 1, 5, 6, 10, 3, 808000, tzinfo=tzlocal())},
 {'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:835319576252:pipeline/dsoaws-1609826255-p-mz9w9bo7oooi/execution/fzfyr23fvab6',
  'PipelineExecutionDisplayName': 'execution-1609826827202',
  'PipelineExecutionStatus': 'Failed',
  'StartTime': datetime.datetime(2021, 1, 5, 6, 7, 7, 92000, tzinfo=tzlocal())}]


# Check Pipeline Executions


In [49]:
# print(sagemaker_project_name_and_id)

# sm.list_pipeline_executions(PipelineName=sagemaker_project_name_and_id)

In [50]:
%store sagemaker_mlops_build_code
%store sagemaker_mlops_deploy_code
%store sagemaker_project_name
%store sagemaker_project_id
%store sagemaker_project_name_and_id
%store sagemaker_project_arn
%store sagemaker_pipeline_product_id
%store sagemaker_pipeline_product_provisioning_artifact_id

Stored 'sagemaker_mlops_build_code' (str)
Stored 'sagemaker_mlops_deploy_code' (str)
Stored 'sagemaker_project_name' (str)
Stored 'sagemaker_project_id' (str)
Stored 'sagemaker_project_name_and_id' (str)
Stored 'sagemaker_project_arn' (str)
Stored 'sagemaker_pipeline_product_id' (str)
Stored 'sagemaker_pipeline_product_provisioning_artifact_id' (str)


In [ ]:
%store